# LSA

잠재 의미 분석(Latent Semantic Analysis, LSA)을 통해 단어 벡터화 기법을 실습하고 단어 간 유사도를 구해볼 것입니다. 여기서의 코드 예제 및 설명은 《밑바닥부터 시작하는 딥러닝 2》를 참고했습니다.

## PTB 데이터 불러오기

**펜 트리뱅크**(Penn Treebank, PTB) 데이터셋. word2vec 발명자인 토마스 미콜로프(Tomas Mikolov) 웹 페이지에서 받을 수 있습니다. 원래의 PTB 문장에 몇 가지 전처리가 되어있습니다. 희소한 단어는 `<unk>`로 치환되어 있다던가, 구체적인 숫자는 `N`으로 대체되어 있습니다.

In [1]:
import os

if 'ptb.train.txt' in os.listdir():
    with open("ptb.train.txt", 'r') as f:
        text = f.read()        
else:
    from urllib.request import urlopen
    url = 'https://raw.githubusercontent.com/tomsercu/lstm/master/data/ptb.train.txt'
    html = urlopen(url)
    text = html.read().decode()

    with open("ptb.train.txt", 'w') as f:
        f.write(text)
    

## 통계기반 벡터화

동시발행 행렬을 만들고, PPMI 행렬로 변환한 다음, 안정성을 높이기 위해 SVD를 이용해 차원을 감소시켜 각 단어의 분산 표현을 만들어냅니다.

### 함수 정의

In [2]:
import numpy as np

# 전처리
def preprocess(text):
    
    words = text.replace('\n', '<eos>').strip().split()
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

# 동시발생 행렬 만들기
def create_co_matrix(corpus, vocab_size, window_size):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size))
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size+1):
            left_idx = idx-i
            right_idx = idx+i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    return co_matrix

# 양의 상호정보량 구하기
def ppmi(C, eps=1e-8):
    N = np.sum(C)
    S = np.sum(C, axis=0, keepdims=True)
    pmi = np.log2(C * N / np.dot(S.T, S) + eps)
    ppmi = np.maximum(pmi, 0)
    return ppmi

### 실행

In [3]:
window_size = 2
wordvec_size = 100

corpus, word_to_id, id_to_word = preprocess(text)
print("동시발생 수 계산 ...")
vocab_size = len(word_to_id)
co_matrix = create_co_matrix(corpus, vocab_size, window_size)
print("PPMI 계산 ...")
W = ppmi(co_matrix)

print("SVD 계산 ...")
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=wordvec_size, algorithm='randomized', n_iter=5)
word_vecs = svd.fit_transform(W) 

# 동일한 다른 방법
# from sklearn.utils.extmath import randomized_svd
# word_vecs, _, _ = randomized_svd(W, n_components=wordvec_size, n_iter=5)

동시발생 수 계산 ...
PPMI 계산 ...
SVD 계산 ...


## 평가

특정 단어를 넣었을 때 유사한 단어를 가려낼 수 있는지, 그리고 단어 간 비유적 관계를 찾아낼 수 있는지를 볼 것입니다.

### 함수 정의

In [4]:
# 가장 비슷한 단어 출력
def most_similar(query, word_to_id, id_to_word, similarity_matirx, top=5):
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return
    
    print('\n[query] ' + query)
    query_id = word_to_id[query]
    similarity = similarity_matirx[query_id]
    
    # 코사인 유사도 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s: %s' % (id_to_word[i], similarity[i]))
        
        count += 1
        if count >= top:
            return


def analogy(a, b, c, word_to_id, id_to_word, word_vecs, top=5):
    for word in (a, b, c):
        if word not in word_to_id:
            print('%s(을)를 찾을 수 없습니다.' % word)
            return

    print('\n[analogy] ' + a + ':' + b + ' = ' + c + ':?')
    a_vec, b_vec, c_vec = word_vecs[word_to_id[a]], word_vecs[word_to_id[b]], word_vecs[word_to_id[c]]
    query_vec = b_vec - a_vec + c_vec
    word_vecs_norm = word_vecs / np.linalg.norm(word_vecs, axis=1, keepdims=True)
    similarity = np.dot(word_vecs_norm, query_vec)

    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] in (a, b, c):
            continue
        print(' {0}: {1}'.format(id_to_word[i], similarity[i]))

        count += 1
        if count >= top:
            return

### 실행

In [5]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matirx = cosine_similarity(word_vecs)

querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, similarity_matirx)


[query] you
 i: 0.8306300170286577
 we: 0.8035908083942216
 do: 0.7708317864744261
 anybody: 0.7187499925464061
 me: 0.714916179863626

[query] year
 month: 0.805700784664732
 earlier: 0.7794736001090208
 last: 0.764942901327093
 quarter: 0.7595937077934993
 next: 0.7352714292723379

[query] car
 auto: 0.7221239761747404
 luxury: 0.6844566161549585
 vehicle: 0.6641306110764477
 domestic: 0.6436364956725468
 cars: 0.6325514759427554

[query] toyota
 motor: 0.765696078978967
 nissan: 0.7279285766332921
 honda: 0.7101727101809763
 lexus: 0.6941262574287386
 mazda: 0.688327613804317


In [6]:
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)


[analogy] king:man = queen:?
 woman: 13.991380407599534
 worker: 12.462211530638415
 boy: 11.3781268479203
 himself: 11.107136042208209
 whom: 10.898577075485491

[analogy] take:took = go:?
 went: 21.543986158254434
 goes: 19.119449644036862
 turns: 18.18632683900878
 ran: 17.23701191060183
 moved: 17.007751976888613

[analogy] car:cars = child:?
 roads: 11.46565419105771
 quantity: 11.462979319796093
 disabled: 11.333230016723235
 hats: 11.14799930309691
 repairs: 11.061422359090107

[analogy] good:better = bad:?
 worse: 14.262228925724756
 significantly: 12.660040328256034
 bigger: 12.586679909336176
 faster: 12.353641162541454
 anyway: 12.265623801513282
